'{"amount":1.0,"base":"EUR","date":"2025-02-10","rates":{"AUD":1.6448,"BGN":1.9558,"BRL":5.9622,"CAD":1.4798,"CHF":0.9396,"CNY":7.5395,"CZK":25.073,"DKK":7.4584,"GBP":0.83283,"HKD":8.04,"HUF":403.88,"IDR":16868,"ILS":3.679,"INR":90.27,"ISK":146.6,"JPY":156.92,"KRW":1496.81,"MXN":21.238,"MYR":4.613,"NOK":11.5745,"NZD":1.8257,"PHP":60.011,"PLN":4.1758,"RON":4.9769,"SEK":11.2765,"SGD":1.3972,"THB":34.993,"TRY":37.148,"USD":1.032,"ZAR":19.0069}}'

In [ ]:
from vertexai.generative_models import (
    Content,
    FunctionDeclaration,
    GenerativeModel,
    Part,
    Tool,
)

model = GenerativeModel("gemini-1.5-pro-001")

get_plot_args = FunctionDeclaration(
    name="plot_line_chart",
    description="Get the exchange rate for currencies between countries",
    parameters={
    "type": "object",
    "properties": {
        "companies": {
            "type": "string",
            "description": "string of company name include [google, facebook, amazon, apple, microsoft]. If user assign other company name, return out of data"
        },
        "index": {
            "type": "integer",
            "description": ""
        },
        "quarters": {
            "type": "string",
            "description": "from Q1 to Q4, if user assign latest, return Q4"
        },
        "years": {
            "type": "integer",
            "description": "from 2020 to 2024, if user assign latest, return 2024"
        },
    },
         "required": [
            "company",
            "index",
            "quarters",
            "years"
      ]
  },
)

plot_tool = Tool(
    function_declarations=[get_plot_args],
)

prompt = """100塊台幣是多少美金?"""

response = model.generate_content(
    prompt,
    tools=[plot_tool],
)

In [8]:
response

candidates {
  content {
    role: "model"
    parts {
      function_call {
        name: "get_exchange_rate"
        args {
          fields {
            key: "currency_to"
            value {
              string_value: "USD"
            }
          }
          fields {
            key: "currency_from"
            value {
              string_value: "TWD"
            }
          }
          fields {
            key: "currency_date"
            value {
              string_value: "latest"
            }
          }
        }
      }
    }
  }
  finish_reason: STOP
  safety_ratings {
    category: HARM_CATEGORY_HATE_SPEECH
    probability: NEGLIGIBLE
    probability_score: 0.121582031
    severity: HARM_SEVERITY_NEGLIGIBLE
    severity_score: 0.0927734375
  }
  safety_ratings {
    category: HARM_CATEGORY_DANGEROUS_CONTENT
    probability: NEGLIGIBLE
    probability_score: 0.291015625
    severity: HARM_SEVERITY_LOW
    severity_score: 0.376953125
  }
  safety_ratings {
    category: 

In [9]:
params = {}
for key, value in response.candidates[0].content.parts[0].function_call.args.items():
    params[key[9:]] = value
params

{'from': 'TWD', 'date': 'latest', 'to': 'USD'}

In [12]:
import requests

url = "https://cdn.jsdelivr.net/npm/@fawazahmed0/currency-api@latest/v1/currencies.json"
api_response = requests.get(url)

In [18]:
api_response.text

'{\n\t"1inch": "1inch",\n\t"aave": "Aave",\n\t"ada": "Cardano",\n\t"aed": "Emirati Dirham",\n\t"afn": "Afghan Afghani",\n\t"agix": "SingularityNET",\n\t"akt": "Akash Network",\n\t"algo": "Algorand",\n\t"all": "Albanian Lek",\n\t"amd": "Armenian Dram",\n\t"amp": "Amp",\n\t"ang": "Dutch Guilder",\n\t"aoa": "Angolan Kwanza",\n\t"ape": "ApeCoin",\n\t"apt": "Aptos",\n\t"ar": "Arweave",\n\t"arb": "Arbitrum",\n\t"ars": "Argentine Peso",\n\t"atom": "Cosmos",\n\t"ats": "Austrian Schilling",\n\t"aud": "Australian Dollar",\n\t"avax": "Avalanche",\n\t"awg": "Aruban or Dutch Guilder",\n\t"axs": "Axie Infinity",\n\t"azm": "Azerbaijani Manat",\n\t"azn": "Azerbaijan Manat",\n\t"bake": "BakeryToken",\n\t"bam": "Bosnian Convertible Mark",\n\t"bat": "Basic Attention Token",\n\t"bbd": "Barbadian or Bajan Dollar",\n\t"bch": "Bitcoin Cash",\n\t"bdt": "Bangladeshi Taka",\n\t"bef": "Belgian Franc",\n\t"bgn": "Bulgarian Lev",\n\t"bhd": "Bahraini Dinar",\n\t"bif": "Burundian Franc",\n\t"bmd": "Bermudian Dollar"

In [10]:
import requests
url = f"https://api.frankfurter.app/{params['date']}"
api_response = requests.get(url, params=params)
api_response.text

'{"message":"not found"}'

In [11]:
response = model.generate_content(
    [
    Content(role="user", parts=[
        Part.from_text(prompt + """Give your answer in steps with lots of detail
            and context, including the exchange rate and date."""),
    ]),
    Content(role="function", parts=[
        Part.from_dict({
            "function_call": {
                "name": "get_exchange_rate",
            }
        })
    ]),
    Content(role="function", parts=[
        Part.from_function_response(
            name="get_exchange_rate",
            response={
                "content": api_response.text,
            }
        )
    ]),
    ],
    tools=[exchange_rate_tool],
)


response.candidates[0].content.parts[0].text

'I can\'t get the exchange rate. The API returned an error: "not found". Therefore, I cannot calculate the conversion from Taiwanese Dollars to US Dollars. \n'